# Update Script
This script will update the nba_db on our postgres server. <br>Looks at our last attempted pull date and pulls every date from then until the current date.

In [39]:
import io
import os
import psycopg2 as pg
from psycopg2 import Error
from sqlalchemy import create_engine
import pandas as pd
import datetime as dt 
import random
from time import sleep


import nba_api.stats.endpoints as ep

#internal scripts
import proj_functions as pf
from connection_data import c_data


### File Locations

In [2]:
loc_scoreboard = '/media/dan/Dan External/nba_data/scoreboard/'
loc_player_bs_trad = '/media/dan/Dan External/nba_data/player_bs_trad/'
loc_player_bs_advanced = '/media/dan/Dan External/nba_data/player_bs_advanced/'
loc_player_bs_ff = '/media/dan/Dan External/nba_data/player_bs_ff/'
loc_player_bs_misc = '/media/dan/Dan External/nba_data/player_bs_misc/'
loc_player_bs_pt = '/media/dan/Dan External/nba_data/player_bs_pt/'
loc_player_bs_scoring = '/media/dan/Dan External/nba_data/player_bs_scoring/'
loc_player_bs_summ = '/media/dan/Dan External/nba_data/player_bs_summ/'
loc_player_bs_tracking = '/media/dan/Dan External/nba_data/player_bs_tracking/'
loc_player_bs_usage = '/media/dan/Dan External/nba_data/player_bs_usage/'

loc_team_bs_trad = '/media/dan/Dan External/nba_data/team_bs_trad/'
loc_team_bs_advanced = '/media/dan/Dan External/nba_data/team_bs_advanced/'
loc_team_bs_ff = '/media/dan/Dan External/nba_data/team_bs_ff/'
loc_team_bs_misc = '/media/dan/Dan External/nba_data/team_bs_misc/'
loc_team_bs_pt = '/media/dan/Dan External/nba_data/team_bs_pt/'
loc_team_bs_scoring = '/media/dan/Dan External/nba_data/team_bs_scoring/'
loc_team_bs_summ = '/media/dan/Dan External/nba_data/team_bs_summ/'
loc_team_bs_tracking = '/media/dan/Dan External/nba_data/team_bs_tracking/'

In [28]:
file_names = ['scoreboard_{}_{}_{}', 'player_bs_trad_{}_{}_{}', 'player_bs_adv_{}_{}_{}',
             'player_bs_ff_{}_{}_{}', 'player_bs_misc_{}_{}_{}', 'player_bs_pt_{}_{}_{}',
             'player_bs_scoring_{}_{}_{}', 'player_bs_summ_{}_{}_{}', 'player_bs_tracking_{}_{}_{}',
             'player_bs_usage_{}_{}_{}', 'team_bs_trad_{}_{}_{}', 'team_bs_adv_{}_{}_{}', 
             'team_bs_ff_{}_{}_{}', 'team_bs_misc_{}_{}_{}', 'team_bs_pt_{}_{}_{}', 
             'team_bs_scoring_{}_{}_{}', 'team_bs_summ_{}_{}_{}', 'team_bs_tracking_{}_{}_{}']
table_names = ['raw.scoreboard', 'raw.player_bs', 'raw.player_bs_adv', 'raw.player_bs_ff',
               'raw.player_bs_misc', 'raw.player_bs_pt', 'raw.player_bs_scoring', 'raw.player_bs_summ',
               'raw.player_bs_tracking', 'raw.player_bs_usage', 'raw.team_bs', 'raw.team_bs_adv', 
               'raw.team_bs_ff', 'raw.team_bs_misc','raw.team_bs_pt', 'raw.team_bs_scoring', 
              'raw.team_bs_summ', 'raw.team_bs_tracking']
save_locations = [loc_scoreboard, loc_player_bs_trad, loc_player_bs_advanced,
                  loc_player_bs_ff, loc_player_bs_misc, loc_player_bs_pt, 
                  loc_player_bs_scoring, loc_player_bs_summ, loc_player_bs_tracking,
                  loc_player_bs_usage, loc_team_bs_trad, loc_team_bs_advanced,
                  loc_team_bs_ff, loc_team_bs_misc, loc_team_bs_pt, loc_team_bs_scoring,
                  loc_team_bs_summ, loc_team_bs_tracking]

## Script

In [5]:
#Start with the last date pulled
last_pull_date = pf.get_last_date()
yesterday = dt.date.today() - dt.timedelta(a)

while last_pull_date < yesterday
    try:
        #add sleep to reduce chances of being blocked
        sleep(random.choices([1, 10, 100], weights=[.58, .41, .01])[0] * random.uniform(.01, 3))
        # pull the scoreboard for the current pull date
        sb = pf.get_scoreboard(current_pull_date)
        if sb == 'no games':
            update_last_date(current_pull_date)
            pass
        gids = sb['GAME_ID']
        # get all dataframes
        df_player_bs_advanced = pf.get_player_bs_advanced(gids)
        df_team_bs_advanced = pf.get_df_team_bs_advanced(gids)

        df_player_bs_ff = pf.get_player_bs_ff(gids)
        df_team_bs_ff = pf.get_team_bs_ff(gids)

        df_player_bs_misc = pf.get_player_bs_misc(gids)
        df_team_bs_misc = pf.get_team_bs_misc(gids)

        df_player_bs_pt = pf.get_player_bs_pt(gids)
        df_team_bs_pt = pf.get_player_bs_pt(gids)
        
        #add a sleep to reduce chances of being blocked
        sleep(random.choices([1, 10, 100], weights=[.58, .41, .01])[0] * random.uniform(.01, 3))

        df_player_bs_scoring = pf.get_player_bs_scoring(gids)
        df_team_bs_scoring = pf.get_team_bs_scoring(gids)

        df_player_bs_summ = pf.get_player_bs_summ(gids)
        df_team_bs_summ = pf.get_team_bs_summ(gids)

        df_player_bs_trad = pf.get_player_bs_trad(gids)
        df_team_bs_trad = pf.get_team_bs_trad(gids)

        df_player_bs_tracking = pf.get_player_bs_tracking(gids)
        df_team_bs_tracking = pf.get_team_bs_tracking(gids)

        df_player_bs_usage = pf.get_player_bs_usage(gids)

        dfs = [sb, df_player_bs_trad, df_player_bs_advanced, df_player_bs_ff,
              df_player_bs_misc, df_player_bs_pt, df_player_bs_scoring,
              df_player_bs_summ, df_player_bs_tracking, df_team_bs_trad,
              df_team_bs_advanced, df_team_bs_ff, df_team_bs_misc, df_team_bs_pt,
              df_team_bs_scoring, df_team_bs_summ, df_team_bs_tracking]

        # Save Dataframes if have data and upload to database
        for df, name, table, location in zip(dfs, file_names, table_names, save_locations):
            df['file_name'] = name.format(current_pull_date.year, current_pull_date.month, current_pull_date.day)
            save_data(df, name, location)
            upload_data(df, table)

        #update the last pull date
        update_last_date(current_pull_date)
        last_pull_date = pf.get_last_date()



    Except:
        print('Error')
        break